In [ ]:
from datetime import date
from pathlib import Path

import ipywidgets as widgets
import yaml
from ipyfilechooser import FileChooser
from matplotlib import cm

from src.landsat import LandsatDownloaderTool, process_products
from src.utils import merge_products, unique_session_id
from src.visualisation import product_visualiser

# Session Setup
Please run the following cells to setup your session. (i.e. Setting up download and processing directories)

You may update the session ID below from the default value to your preferred value.

In [ ]:
session_id = widgets.Text(description="Session ID: ", value=unique_session_id())
display(session_id)

In [ ]:
# Load Config File
with open("config.yaml", "r") as file:
    config = yaml.safe_load(file)

In [ ]:
# Load Directory Paths and Create Directories
raw_dir = Path(config["LANDSAT_RAW"], session_id.value)
processed_dir = Path(config["LANDSAT_PROCESSED"], session_id.value)

raw_dir.mkdir(parents=True, exist_ok=True)
processed_dir.mkdir(parents=True, exist_ok=True)

# Search for Landsat 8/9 Products

Please enter your Landsat search criteria below. You may either define a bounding box or upload a shape file.

**NOTE**: Landsat products can be downloaded via Digital Earth Australia's (DEA) S3 Bucket or via NCI THREDDS. Currently, DEA is offering the products for free but may change to a "requestor pays" model in the future.

In [ ]:
start_date = widgets.DatePicker(description="Start Date: ", value=date(2024, 1, 1))
end_date = widgets.DatePicker(description="End Date: ", value=date(2024, 1, 8))

min_lon = widgets.FloatText(description="Min Lon: ", value=116.01897)
min_lon.style.description_width = "initial"

max_lon = widgets.FloatText(description="Max Lon: ", value=116.20093)
max_lon.style.description_width = "initial"

min_lat = widgets.FloatText(description="Min Lat: ", value=-32.30959)
min_lat.style.description_width = "initial"

max_lat = widgets.FloatText(description="Max Lat: ", value=-31.98176)
max_lat.style.description_width = "initial"

shapefile_path = FileChooser(
    title="Upload a Shapefile (Optional): ", filter_pattern="*.shp"
)

collections = widgets.Select(
    options={"Landsat-8": "ga_ls8c_ard_3", "Landsat-9": "ga_ls9c_ard_3"},
    description="Select Landsat collection: ",
    rows=2,
    value="ga_ls8c_ard_3",
)
collections.style.description_width = "initial"

download_from_thredds = widgets.Checkbox(
    value=False, description="Download from NCI THREDDS? ", indent=False
)

search_widgets = widgets.VBox(
    [
        start_date,
        end_date,
        widgets.HTML("<br>"),
        widgets.Label(
            "Please enter your area of interest coordinates or upload a Shapefile."
        ),
        min_lon,
        max_lon,
        min_lat,
        max_lat,
        shapefile_path,
        widgets.HTML("<br>"),
        collections,
        download_from_thredds,
    ]
)

display(search_widgets)

In [ ]:
# Initialise Client
client = LandsatDownloaderTool()

# Store Search Parameters
search_params = client.search_parameters(
    start_date=start_date.value,
    end_date=end_date.value,
    min_lon=min_lon.value,
    max_lon=max_lon.value,
    min_lat=min_lat.value,
    max_lat=max_lat.value,
    shapefile=shapefile_path.selected,
    collection=collections.value,
)

# Search for Products from DEA STAC API
search_results = client.search(search_params, True)

In [ ]:
# Return next aquisition date for AOI (Approximate)
client.get_next_aquisition_date(search_params)

# Download Landsat 8/9 Products

Please confirm your search results above. Depending on your search criteria, there may be a large number of products to download. 

Please be patient.

The filenames of downloaded Level 2 products have the follow naming convention (described in more detail [here](https://knowledge.dea.ga.gov.au/guides/reference/collection_3_naming/)).

![image](notebook_images/ARD_Landsat_Filename.svg)  
Image © Commonwealth of Australia (Geoscience Australia) 2021, [Digital Earth Austraia](https://knowledge.dea.ga.gov.au/guides/reference/collection_3_naming/), [CC BY 4.0](https://www.ga.gov.au/copyright)

In [ ]:
# Download Search Results
client.download(
    search_results=search_results,
    download_dir=raw_dir,
    download_from_thredds=download_from_thredds.value,
)

# NDVI, NDMI and NBR Processing

Please select which Landsat product(s) to process. (Use Ctrl to select multiple).
You have the option of processing all downloaded products, but it may take some time.

In [ ]:
# Retrieve all downloaded Landsat directories
landsat_dirs = [dir for dir in list(raw_dir.glob("*")) if dir.is_dir()]

selected_products = widgets.SelectMultiple(
    options=landsat_dirs,
    rows=10,
    description="Select a Landsat product(s) to process: ",
    layout=widgets.Layout(width="auto"),
)
selected_products.style.description_width = "initial"

select_all_products = widgets.Checkbox(
    value=False, description="Select all products? ", indent=False
)

crop = widgets.Checkbox(
    value=False, description="Crop to Area of Interest? ", indent=False
)

processing_widgets = widgets.VBox([selected_products, select_all_products, crop])
display(processing_widgets)

In [ ]:
process_products(
    product_paths=selected_products.value,
    landsat_dirs=landsat_dirs,
    landsat_processed_dir=processed_dir,
    all_products=select_all_products.value,
    crop=crop.value,
    bbox=(min_lon.value, min_lat.value, max_lon.value, max_lat.value),
    shapefile=shapefile_path.selected,
)

# Merge Processed Products (Optional)
In some cases, the satellite image scenes may not coincide with your selected area of interest. You may use the cells below to select some processed products and merge the raster files. This may be useful for visualising large areas of interest.

In [ ]:
processed_dirs = [dir for dir in list(processed_dir.glob("*")) if dir.is_dir()]

selected_products = widgets.SelectMultiple(
    options=processed_dirs,
    rows=10,
    description="Select satellite products to merge: ",
    layout=widgets.Layout(width="auto"),
)
selected_products.style.description_width = "initial"

display(selected_products)

In [ ]:
merge_products(product_paths=selected_products.value)

# Visualisation
Please select a processed Landsat product to visualise via ipyleaflet. (Use Ctrl to select multiple).

You have the option to select a colormap for your raster layers.

Note: You WILL need to run the %%javascript cell below before displaying the map.

In [ ]:
processed_dirs = [dir for dir in list(processed_dir.glob("*")) if dir.is_dir()]

selected_products = widgets.SelectMultiple(
    options=processed_dirs,
    rows=10,
    description="Select a Sentinel-2 product to visualise: ",
    layout=widgets.Layout(width="auto"),
)
selected_products.style.description_width = "initial"

selected_resample = widgets.Checkbox(
    value=True, 
    description="(RECOMMENDED) Apply a downscaling factor to reduce the spatial resolution of product for rapid display?", 
    indent=False,
    layout=widgets.Layout(width="auto"),
)


resampling_factor = widgets.IntText(
    value=6, description="Downscaling Factor: ", disabled=True
)
resampling_factor.style.description_width = "initial"

description_box = widgets.HTML(
    value="It is recommended to do any actual analysis in python outside of this notebook or in a GIS program such as QGIS/ArcGIS",
    placeholder='',
    description='Note:',
)

selected_colormap = widgets.RadioButtons(
    options={"Viridis": cm.viridis, "Jet": cm.jet, "Turbo": cm.turbo, },
    description="Select a colormap: ",
    layout=widgets.Layout(width="auto"),
)

selected_percentile = widgets.Checkbox(
    value=False, 
    description="Apply Histogram-filter to colormap, potentially enhancing contrast and improve image display?", 
    indent=False,
    layout=widgets.Layout(width="auto"),
)

colormap_fname = 'notebook_images/colormap_preview.png'
with open(colormap_fname, 'rb') as file:
    image = file.read()
colormap_image_widget = widgets.Image(
    value=image,
    format='png',
    width=600
)

visualisation_widgets = widgets.VBox(
    [
        selected_products,
        selected_resample,
        resampling_factor,
        description_box,
        selected_colormap,
        selected_percentile,
        colormap_image_widget
    ]
)
display(visualisation_widgets)

In [ ]:
%%javascript
// Check that we havent already applied the fix. If we havent, apply the fix.
if (window.pixFix === undefined) { 
 window.pixFix = document.createElement('style');
 window.pixFix.innerText = '.leaflet-image-layer { image-rendering: pixelated }';
 console.log(window.pixFix)
 document.head.appendChild(window.pixFix);
}

In [ ]:
product_visualiser(
    product_paths=selected_products.value,
    product_type="DEA",
    apply_resample=selected_resample.value,
    resample_factor=resampling_factor.value,
    use_percentile=selected_percentile.value,
    cmap=selected_colormap.value,
)